Тестовое задание для компании BELTEL.

Выполнил:
Филипенко Вадим
filipenko.vad@gmail.com
89119526959

In [1]:
import os
import pandas as pd
import json

Алгоритм парсинга для одного файла:
1. Загрузить файл json как словарь.
2. Создать словарь для хранения всей извлечённой из файла информации (добавить необходимые ключи, если их нет).
3. Считать из преобразованного файла json общую информацию о файле(Filename, DateBegin, DateEnd, PWCcode и количество элементов в массиве GoodsLists).
4. Для каждого элемента массива GoodsLists (перебор элементов по range(0,qty_goods_lists)) вытащить информацию о DiscountType, DiscountValue, Value, FirstValue, LessOrEqua. Сохранить эту информацию в переменную, обработав исключения (в случае отсутствия поля, вставлять None).
5. Считать количество элементов в массиве Prices.
6. Для каждого элемента массива Prices (перебор элементов по range(0, QtyPrices)) извлечь "Data" и внутри этого списка, с помощью  перебора по QtyGoods извлечь 4 столбца "Item", "SalePriceBeforePromo", "SalePriceTimePromo", "DatePriceBeforePromo", а дальше добавляем их к остальным извлечённым столбцам в итоговый словарь, хранящий все результаты для каждого из элементов "Data".

In [2]:
# Функция для извлечения данных из JSON файла
def parse_promo_json(file_to_parse, result, filename):
    '''
    Извлекает информацию из JSON файла определённого формата и сохраняет в словарь.

    :param json_file: Путь к файлу, из которого необходимо извлечь информацию.
    :param result: Словарь, в который записываются извлечённые данные.
    :param filename: Имя файла из которого извлекают данные
    :return: Возвращает словарь с извлечёнными данными
    '''
    with open(file_to_parse) as f:
        json_file = json.load(f)

    #Создание необходимых ключей в словаре, если их нет
    columns = ['Item', 'SalePriceBeforePromo', 'SalePriceTimePromo',
               'DatePriceBeforePromo', 'ObjCode', 'DiscountType', 'DiscountValue',
               'DateBegin', 'DateEnd', 'PWCcode', 'Value', 'FirstValue', 'LessOrEqual',
               'File']
    for column in columns:
        if column not in result:
            result[column] = []

    #Данные, общие для всего файла
    date_begin = json_file['GeneralInfo']['DateBegin']
    date_end = json_file['GeneralInfo']['DateEnd']
    pwc_code = json_file['GeneralInfo']['PWCcode']
    #Количество списков товаров в файле
    qty_goods_lists = len(json_file['Information']['GoodsLists'])
    #Циклы для считывания данных из файла
    for i in range(0, qty_goods_lists):
            try:
                #Проверка переменных на присутствие значений в файлах
                if 'DiscountType' in json_file['Information']['GoodsLists'][i]:
                    discount_type = json_file['Information']['GoodsLists'][i]['DiscountType']
                else:
                    discount_type = None
                if 'DiscountValue' in json_file['Information']['GoodsLists'][i]:
                    discount_value = json_file['Information']['GoodsLists'][i]['DiscountValue']
                else:
                    discount_value = None
                if 'Value' in json_file['Information']['GoodsLists'][i]['PriceOptions'][0]:
                    value = json_file['Information']['GoodsLists'][i]['PriceOptions'][0]['Value']
                else:
                    value = None
                if 'FirstValue' in json_file['Information']['GoodsLists'][i]['PriceOptions'][0]:
                    first_value = json_file['Information']['GoodsLists'][i]['PriceOptions'][0]['FirstValue']
                else:
                    first_value = None
                if 'Operator' in json_file['Information']['GoodsLists'][i]['PriceOptions'][0]:
                    less_or_equal = json_file['Information']['GoodsLists'][i]['PriceOptions'][0]['Operator']
                else:
                    less_or_equal = None
            except TypeError:
                pass
            qty_prices = json_file['Information']['GoodsLists'][i]['QtyPrices']
            for j in range(0,qty_prices):
                qty_goods = json_file['Information']['GoodsLists'][i]['Prices'][j]['QtyGoods']
                for k in range(0, qty_goods):
                    result["Item"].append(json_file['Information']['GoodsLists'][i]['Prices'][j]['Data'][k][0])
                    result["SalePriceBeforePromo"].append(json_file['Information']['GoodsLists'][i]['Prices'][j]['Data'][k][1])
                    result["SalePriceTimePromo"].append(json_file['Information']['GoodsLists'][i]['Prices'][j]['Data'][k][2])
                    result["DatePriceBeforePromo"].append(json_file['Information']['GoodsLists'][i]['Prices'][j]['Data'][k][3])
                    result["ObjCode"].append(json_file['Information']['GoodsLists'][i]['Prices'][j]['StoreCode'])
                    result["DiscountType"].append(discount_type)
                    result["DiscountValue"].append(discount_value)
                    result["DateBegin"].append(date_begin)
                    result["DateEnd"].append(date_end)
                    result["PWCcode"].append(pwc_code)
                    result["Value"].append(value)
                    result["FirstValue"].append(first_value)
                    result["LessOrEqual"].append(less_or_equal)
                    result['File'].append(filename)
    return result

In [3]:
#Считаем текущую рабочую директорию
directory = os.getcwd()

#Сохраним список названий json файлов с информацией
filenames = [f for f in os.listdir(directory+'/promo') if f.startswith('export')]

#Создадим пустой словарь для хранения извлечённых значений
parsed_files  = {}

#Цикл для считывания всех файлов в директории 'promo'
for filename in filenames:
    file = './promo/' + filename
    parse_promo_json(file_to_parse=file, result=parsed_files, filename=filename)

In [4]:
#Создадим датафрейм из извлечённых данных
df = pd.DataFrame.from_dict(parsed_files).fillna('None')
#Сохраним датафрейм в формате .xlsx
df.to_excel('./result.xlsx', sheet_name='Promo', index=False)

In [5]:
df.head()

,Item,SalePriceBeforePromo,SalePriceTimePromo,DatePriceBeforePromo,ObjCode,DiscountType,DiscountValue,DateBegin,DateEnd,PWCcode,Value,FirstValue,LessOrEqual,File
0,175164,64.9,34.9,2021-01-28T00:00:00,1303,PricePerUnit,34.9,2020-01-04T18:40:38,2020-01-08T23:59:00,000006632,None,2,MoreOrEqual,export_20210913-1652_1223_6174.json
1,175163,62.9,34.9,2021-07-15T00:00:00,1303,PricePerUnit,34.9,2020-01-04T18:40:38,2020-01-08T23:59:00,000006632,None,2,MoreOrEqual,export_20210913-1652_1223_6174.json
2,100487127,62.9,34.9,2021-07-15T00:00:00,1303,PricePerUnit,34.9,2020-01-04T18:40:38,2020-01-08T23:59:00,000006632,None,2,MoreOrEqual,export_20210913-1652_1223_6174.json
3,100122494,62.9,34.9,2021-07-15T00:00:00,1303,PricePerUnit,34.9,2020-01-04T18:40:38,2020-01-08T23:59:00,000006632,None,2,MoreOrEqual,export_20210913-1652_1223_6174.json
4,100357074,62.9,34.9,2021-07-15T00:00:00,1303,PricePerUnit,34.9,2020-01-04T18:40:38,2020-01-08T23:59:00,000006632,None,2,MoreOrEqual,export_20210913-1652_1223_6174.json


In [6]:
print(f'Количество извлечённых записей - {df.shape[0]}')

Количество извлечённых записей - 30534
